# Introduction
Processing of datasets in/around Rio de Janeiro

### Imports
Import libraries and write settings here.

In [1]:
#import os

# Data manipulation
import pandas as pd
import numpy as np
import regex as re

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

<img src="../../../images/zapimoveis-sample.jpg" alt="Sample listing from the zapmoveis (BR) page" height="369" width="684"/>

# Data Imports
- Let **rio1_df** be data from [ZAPIMOVEIS - Rio de Janeiro - BR](http://bit.ly/37ovyXb)
    - **R\$** is the Brazilian real.
    - **IPTU** is a municipal tax charged over the property of urban real estate.

In [2]:
rio1_df = pd.read_csv('./data.csv', header='infer')

In [3]:
rio1_df.shape

(7680, 10)

In [4]:
print(rio1_df.columns)

Index(['Unnamed: 0', 'address', 'areas', 'bathrooms', 'bedrooms',
       'parking-spaces', 'price', 'price-item-0', 'price-item-1',
       'Unnamed: 9'],
      dtype='object')


In [5]:
# Check
rio1_df

,Unnamed: 0,address,areas,bathrooms,bedrooms,parking-spaces,price,price-item-0,price-item-1,Unnamed: 9
0,0,"Rua João Barros Moreira, Recreio dos Bandeirantes",450 m²,7.0,4.0,3.0,R$ 5.000 /mês,condomínio R$ 1.242,IPTU R$ 481,NaN
1,1,"Rua João Pinheiro, Piedade",56 m²,2.0,2.0,1.0,R$ 1.300 /mês,condomínio R$ 460,IPTU R$ 60,NaN
2,2,"Estrada dos Bandeirantes, Jacarepaguá",61 m²,2.0,2.0,1.0,R$ 1.380 /mês,condomínio R$ 600,IPTU R$ 89,NaN
3,3,"Rua Mário Covas Júnior, Barra da Tijuca",97 m²,3.0,2.0,1.0,R$ 2.950 /mês,condomínio R$ 852,IPTU R$ 315,NaN
4,4,"Rua Coronel Paulo Malta Rezende, Barra da Tijuca",65 m²,2.0,2.0,1.0,R$ 2.400 /mês,condomínio R$ 809,IPTU R$ 218,NaN
...,...,...,...,...,...,...,...,...,...,...
7675,29,"Rua Professor Manuel Ferreira, Gávea",85 m²,2.0,2.0,1.0,R$ 3.500 /mês,condomínio R$ 990,IPTU R$ 306,NaN
7676,30,"Rua General Urquiza, Leblon",57 m²,1.0,2.0,1.0,R$ 3.300 /mês,condomínio R$ 1.000,IPTU R$ 270,NaN
7677,32,"Avenida Lúcio Costa, Barra da Tijuca",290 m²,5.0,4.0,3.0,R$ 18.000 /mês,condomínio R$ 5.400,IPTU R$ 2.600,NaN
7678,33,"Rua Santa Alexandrina, Rio Comprido",58 m²,1.0,1.0,NaN,R$ 1.200 /mês,condomínio R$ 566,IPTU R$ 15,NaN


In [6]:
# Drop unnecessary index column in Rio1 CSV
rio1_df.drop(columns=['Unnamed: 0'], inplace=True)

In [7]:
# Proportions of RE data missing
round(rio1_df.isna().mean() * 100, 2)

address            0.00
areas             11.41
bathrooms         19.26
bedrooms           0.10
parking-spaces    35.13
price              0.00
price-item-0       2.94
price-item-1      15.73
Unnamed: 9        88.61
dtype: float64

In [8]:
# What sort of data is in col `Unnamed: 9`?
rio1_df["Unnamed: 9"].unique()

array([nan, '3', '2', '1', '6', '55 m²', '4', '56 m²', '73 m²', '75 m²',
       '58 m²', '60 m²', '5', '40 m²', '50 m²', '43 m²', '80 m²', '65 m²',
       '49 m²', '35 m²', '46 m²', '59 m²', '85 m²', '7', '45 m²', '53 m²',
       '235 m²', '180 m²', '8', '95 m²', '70 m²'], dtype=object)

In [9]:
# Drop unnecessary `Unnamed: 9` column in Rio1 CSV
rio1_df.drop(columns=['Unnamed: 9'], inplace=True)

In [10]:
# Data types received
rio1_df.dtypes

address            object
areas              object
bathrooms         float64
bedrooms          float64
parking-spaces    float64
price              object
price-item-0       object
price-item-1       object
dtype: object

In [11]:
# Parse `price`; return new col `x_price` type: float
rio1_df["x_price"] = rio1_df["price"].str.replace('.','').str.extract('(\d+)', expand=True).astype("float")

In [12]:
rio1_df.head()

,address,areas,bathrooms,bedrooms,parking-spaces,price,price-item-0,price-item-1,x_price
0,"Rua João Barros Moreira, Recreio dos Bandeirantes",450 m²,7.0,4.0,3.0,R$ 5.000 /mês,condomínio R$ 1.242,IPTU R$ 481,5000.0
1,"Rua João Pinheiro, Piedade",56 m²,2.0,2.0,1.0,R$ 1.300 /mês,condomínio R$ 460,IPTU R$ 60,1300.0
2,"Estrada dos Bandeirantes, Jacarepaguá",61 m²,2.0,2.0,1.0,R$ 1.380 /mês,condomínio R$ 600,IPTU R$ 89,1380.0
3,"Rua Mário Covas Júnior, Barra da Tijuca",97 m²,3.0,2.0,1.0,R$ 2.950 /mês,condomínio R$ 852,IPTU R$ 315,2950.0
4,"Rua Coronel Paulo Malta Rezende, Barra da Tijuca",65 m²,2.0,2.0,1.0,R$ 2.400 /mês,condomínio R$ 809,IPTU R$ 218,2400.0


In [13]:
# Split/subset the dataframe - take top 10%
# Note: This is for demonstration purposes as the following step submits to external web service GeoPy!
chunk_size = int(rio1_df.shape[0] / 10)
df = rio1_df.iloc[0:0 + chunk_size]
df.shape
df.head()

,address,areas,bathrooms,bedrooms,parking-spaces,price,price-item-0,price-item-1,x_price
0,"Rua João Barros Moreira, Recreio dos Bandeirantes",450 m²,7.0,4.0,3.0,R$ 5.000 /mês,condomínio R$ 1.242,IPTU R$ 481,5000.0
1,"Rua João Pinheiro, Piedade",56 m²,2.0,2.0,1.0,R$ 1.300 /mês,condomínio R$ 460,IPTU R$ 60,1300.0
2,"Estrada dos Bandeirantes, Jacarepaguá",61 m²,2.0,2.0,1.0,R$ 1.380 /mês,condomínio R$ 600,IPTU R$ 89,1380.0
3,"Rua Mário Covas Júnior, Barra da Tijuca",97 m²,3.0,2.0,1.0,R$ 2.950 /mês,condomínio R$ 852,IPTU R$ 315,2950.0
4,"Rua Coronel Paulo Malta Rezende, Barra da Tijuca",65 m²,2.0,2.0,1.0,R$ 2.400 /mês,condomínio R$ 809,IPTU R$ 218,2400.0


In [14]:
# Add geocodes
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="GA_GeoCoder")

# Invoke rate-limited geocode
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Now create location column
df['location'] = df['address'].apply(lambda x : x + ', Brazil').apply(geocode)

# Credit: [GeoPy](https://geopy.readthedocs.io/en/stable/)
#         [towardsdatascience/geocode-with-python](http://bit.ly/2ueKREg)

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [16]:
# Create longitude, latitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

# Split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [17]:
# Check process
df.head()

,address,areas,bathrooms,bedrooms,parking-spaces,price,price-item-0,price-item-1,x_price,location,point,latitude,longitude,altitude
0,"Rua João Barros Moreira, Recreio dos Bandeirantes",450 m²,7.0,4.0,3.0,R$ 5.000 /mês,condomínio R$ 1.242,IPTU R$ 481,5000.0,"(Rua João Barros Moreira, Recreio dos Bandeira...","(-23.0197394, -43.4600439, 0.0)",-23.019739,-43.460044,0.0
1,"Rua João Pinheiro, Piedade",56 m²,2.0,2.0,1.0,R$ 1.300 /mês,condomínio R$ 460,IPTU R$ 60,1300.0,"(Rua João Pinheiro, Piedade, Zona Norte do Rio...","(-22.8871743, -43.3041246, 0.0)",-22.887174,-43.304125,0.0
2,"Estrada dos Bandeirantes, Jacarepaguá",61 m²,2.0,2.0,1.0,R$ 1.380 /mês,condomínio R$ 600,IPTU R$ 89,1380.0,"(Estrada dos Bandeirantes, Jacarepaguá, Zona O...","(-22.9548472, -43.3765899, 0.0)",-22.954847,-43.376590,0.0
3,"Rua Mário Covas Júnior, Barra da Tijuca",97 m²,3.0,2.0,1.0,R$ 2.950 /mês,condomínio R$ 852,IPTU R$ 315,2950.0,"(Rua Mário Covas Júnior, Barra da Tijuca, Zona...","(-23.0056458, -43.3301883, 0.0)",-23.005646,-43.330188,0.0
4,"Rua Coronel Paulo Malta Rezende, Barra da Tijuca",65 m²,2.0,2.0,1.0,R$ 2.400 /mês,condomínio R$ 809,IPTU R$ 218,2400.0,"(Rua Coronel Paulo Malta Rezende, Condomínio S...","(-23.0049338, -43.3312167, 0.0)",-23.004934,-43.331217,0.0


In [18]:
# Unnecessary columns
df.drop(['location'],axis=1,inplace=True)

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
# Unnecessary columns
df.drop(['point'],axis=1,inplace=True)

In [20]:
# Unnecessary columns
df.drop(['altitude'],axis=1,inplace=True)

In [21]:
# Rename for better identity of cols
df.rename(columns={'price':'zap_price_pmo', 'price-item-0':'zap_price_cls','price-item-1':'zap_price_iptu', 'x_price':'r$_pmo'}, inplace=True)

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [22]:
# Convert/calculate a new column from Brazilian real to USD as of 2020-02-16
# 100 r$ = 23.24446200692685 usd
df['usd_pmo'] = [round(row * 0.232446, 2) for row in df['r$_pmo']]

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
# Check process
df.head()

,address,areas,bathrooms,bedrooms,parking-spaces,zap_price_pmo,zap_price_cls,zap_price_iptu,r$_pmo,latitude,longitude,usd_pmo
0,"Rua João Barros Moreira, Recreio dos Bandeirantes",450 m²,7.0,4.0,3.0,R$ 5.000 /mês,condomínio R$ 1.242,IPTU R$ 481,5000.0,-23.019739,-43.460044,1162.23
1,"Rua João Pinheiro, Piedade",56 m²,2.0,2.0,1.0,R$ 1.300 /mês,condomínio R$ 460,IPTU R$ 60,1300.0,-22.887174,-43.304125,302.18
2,"Estrada dos Bandeirantes, Jacarepaguá",61 m²,2.0,2.0,1.0,R$ 1.380 /mês,condomínio R$ 600,IPTU R$ 89,1380.0,-22.954847,-43.376590,320.78
3,"Rua Mário Covas Júnior, Barra da Tijuca",97 m²,3.0,2.0,1.0,R$ 2.950 /mês,condomínio R$ 852,IPTU R$ 315,2950.0,-23.005646,-43.330188,685.72
4,"Rua Coronel Paulo Malta Rezende, Barra da Tijuca",65 m²,2.0,2.0,1.0,R$ 2.400 /mês,condomínio R$ 809,IPTU R$ 218,2400.0,-23.004934,-43.331217,557.87


In [24]:
df['latitude'].value_counts(dropna=False)
df['longitude'].value_counts(dropna=False)

 NaN          63
-43.407209    16
-43.188297     8
-43.371319     8
-43.366280     7
              ..
-43.350948     1
-43.485306     1
-43.175335     1
-43.461767     1
-43.318172     1
Name: longitude, Length: 442, dtype: int64

In [25]:
# Return the set of records missing the lat and long values
no_latlong = df[df['latitude'].isnull() | df['longitude'].isnull()]

In [26]:
# Split address into street and neighborhood
no_latlong[['street', 'neighborhood']] = df['address'].str.split(pat=',',expand=True)

In [27]:
no_latlong.head(10)
no_latlong.shape

(63, 14)

In [28]:
# Add geocodes to neighborhood of MISSING
# from geopy.geocoders import Nominatim
# from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="GA_GeoCoder")

# Invoke rate-limited geocode
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Now create location column
no_latlong['location'] = no_latlong['neighborhood'].apply(lambda x : x + ', Brazil').apply(geocode)

# Credit: [GeoPy](https://geopy.readthedocs.io/en/stable/)
#         [towardsdatascience/geocode-with-python](http://bit.ly/2ueKREg

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [29]:
# Create longitude, latitude and altitude from location column (returns tuple)
no_latlong['point'] = no_latlong['location'].apply(lambda loc: tuple(loc.point) if loc else None)

# Split point column into latitude, longitude and altitude columns
no_latlong[['latitude', 'longitude', 'altitude']] = pd.DataFrame(no_latlong['point'].tolist(), index=no_latlong.index)


/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
# Pare down to the essential missing cols
point_idx_df = no_latlong[['point', 'longitude', 'latitude']]

In [31]:
# Check process
point_idx_df.head()

,point,longitude,latitude
9,"(-22.9408899, -43.3422564, 0.0)",-43.342256,-22.940890
10,"(-22.928988, -43.3534191, 0.0)",-43.353419,-22.928988
14,"(-22.9408899, -43.3422564, 0.0)",-43.342256,-22.940890
33,"(-22.928988, -43.3534191, 0.0)",-43.353419,-22.928988
47,"(-22.8326425, -43.2842228, 0.0)",-43.284223,-22.832642


In [32]:
# Merging in the missed entries to a copy of the rio1 EDA dataset
tmp_df = pd.merge(df, point_idx_df, right_index=True, left_index=True, how="left", suffixes=('_left', '_right'))

In [33]:
tmp_df.head()

,address,areas,bathrooms,bedrooms,parking-spaces,zap_price_pmo,zap_price_cls,zap_price_iptu,r$_pmo,latitude_left,longitude_left,usd_pmo,point,longitude_right,latitude_right
0,"Rua João Barros Moreira, Recreio dos Bandeirantes",450 m²,7.0,4.0,3.0,R$ 5.000 /mês,condomínio R$ 1.242,IPTU R$ 481,5000.0,-23.019739,-43.460044,1162.23,NaN,NaN,NaN
1,"Rua João Pinheiro, Piedade",56 m²,2.0,2.0,1.0,R$ 1.300 /mês,condomínio R$ 460,IPTU R$ 60,1300.0,-22.887174,-43.304125,302.18,NaN,NaN,NaN
2,"Estrada dos Bandeirantes, Jacarepaguá",61 m²,2.0,2.0,1.0,R$ 1.380 /mês,condomínio R$ 600,IPTU R$ 89,1380.0,-22.954847,-43.376590,320.78,NaN,NaN,NaN
3,"Rua Mário Covas Júnior, Barra da Tijuca",97 m²,3.0,2.0,1.0,R$ 2.950 /mês,condomínio R$ 852,IPTU R$ 315,2950.0,-23.005646,-43.330188,685.72,NaN,NaN,NaN
4,"Rua Coronel Paulo Malta Rezende, Barra da Tijuca",65 m²,2.0,2.0,1.0,R$ 2.400 /mês,condomínio R$ 809,IPTU R$ 218,2400.0,-23.004934,-43.331217,557.87,NaN,NaN,NaN


In [34]:
# Pull over the "found" entry on None for latitude
tmp_df['latitude_left'] = np.where(np.isnan(tmp_df['latitude_left']), tmp_df['latitude_right'], tmp_df['latitude_left'])

In [35]:
# Pull over the "found" entry on None for longitude
tmp_df['longitude_left'] = np.where(np.isnan(tmp_df['longitude_left']), tmp_df['longitude_right'], tmp_df['longitude_left'])

In [36]:
# Check process
tmp_df

,address,areas,bathrooms,bedrooms,parking-spaces,zap_price_pmo,zap_price_cls,zap_price_iptu,r$_pmo,latitude_left,longitude_left,usd_pmo,point,longitude_right,latitude_right
0,"Rua João Barros Moreira, Recreio dos Bandeirantes",450 m²,7.0,4.0,3.0,R$ 5.000 /mês,condomínio R$ 1.242,IPTU R$ 481,5000.0,-23.019739,-43.460044,1162.23,NaN,NaN,NaN
1,"Rua João Pinheiro, Piedade",56 m²,2.0,2.0,1.0,R$ 1.300 /mês,condomínio R$ 460,IPTU R$ 60,1300.0,-22.887174,-43.304125,302.18,NaN,NaN,NaN
2,"Estrada dos Bandeirantes, Jacarepaguá",61 m²,2.0,2.0,1.0,R$ 1.380 /mês,condomínio R$ 600,IPTU R$ 89,1380.0,-22.954847,-43.376590,320.78,NaN,NaN,NaN
3,"Rua Mário Covas Júnior, Barra da Tijuca",97 m²,3.0,2.0,1.0,R$ 2.950 /mês,condomínio R$ 852,IPTU R$ 315,2950.0,-23.005646,-43.330188,685.72,NaN,NaN,NaN
4,"Rua Coronel Paulo Malta Rezende, Barra da Tijuca",65 m²,2.0,2.0,1.0,R$ 2.400 /mês,condomínio R$ 809,IPTU R$ 218,2400.0,-23.004934,-43.331217,557.87,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,"Avenida Jaime Poggi, Jacarepaguá",80 m²,2.0,3.0,1.0,R$ 2.200 /mês,condomínio R$ 900,IPTU R$ 150,2200.0,-22.968166,-43.391009,511.38,NaN,NaN,NaN
764,"Rio de Janeiro, Rio de Janeiro",78 m²,2.0,3.0,1.0,R$ 3.000 /mês,condomínio R$ 900,IPTU R$ 100,3000.0,-22.911014,-43.209373,697.34,NaN,NaN,NaN
765,"Rua Severino Ferreira da Costa, Campo Grande",65 m²,1.0,2.0,NaN,R$ 850 /mês,condomínio R$ 250,NaN,850.0,-22.923988,-43.547193,197.58,NaN,NaN,NaN
766,"Estrada do Monteiro, Campo Grande",55 m²,2.0,2.0,1.0,R$ 940 /mês,condomínio R$ 590,IPTU R$ 36,940.0,-22.928051,-43.572791,218.50,NaN,NaN,NaN


In [37]:
# Drop merge columns
tmp_df[tmp_df.columns.drop(['point', 'longitude_right', 'latitude_right'])]

,address,areas,bathrooms,bedrooms,parking-spaces,zap_price_pmo,zap_price_cls,zap_price_iptu,r$_pmo,latitude_left,longitude_left,usd_pmo
0,"Rua João Barros Moreira, Recreio dos Bandeirantes",450 m²,7.0,4.0,3.0,R$ 5.000 /mês,condomínio R$ 1.242,IPTU R$ 481,5000.0,-23.019739,-43.460044,1162.23
1,"Rua João Pinheiro, Piedade",56 m²,2.0,2.0,1.0,R$ 1.300 /mês,condomínio R$ 460,IPTU R$ 60,1300.0,-22.887174,-43.304125,302.18
2,"Estrada dos Bandeirantes, Jacarepaguá",61 m²,2.0,2.0,1.0,R$ 1.380 /mês,condomínio R$ 600,IPTU R$ 89,1380.0,-22.954847,-43.376590,320.78
3,"Rua Mário Covas Júnior, Barra da Tijuca",97 m²,3.0,2.0,1.0,R$ 2.950 /mês,condomínio R$ 852,IPTU R$ 315,2950.0,-23.005646,-43.330188,685.72
4,"Rua Coronel Paulo Malta Rezende, Barra da Tijuca",65 m²,2.0,2.0,1.0,R$ 2.400 /mês,condomínio R$ 809,IPTU R$ 218,2400.0,-23.004934,-43.331217,557.87
...,...,...,...,...,...,...,...,...,...,...,...,...
763,"Avenida Jaime Poggi, Jacarepaguá",80 m²,2.0,3.0,1.0,R$ 2.200 /mês,condomínio R$ 900,IPTU R$ 150,2200.0,-22.968166,-43.391009,511.38
764,"Rio de Janeiro, Rio de Janeiro",78 m²,2.0,3.0,1.0,R$ 3.000 /mês,condomínio R$ 900,IPTU R$ 100,3000.0,-22.911014,-43.209373,697.34
765,"Rua Severino Ferreira da Costa, Campo Grande",65 m²,1.0,2.0,NaN,R$ 850 /mês,condomínio R$ 250,NaN,850.0,-22.923988,-43.547193,197.58
766,"Estrada do Monteiro, Campo Grande",55 m²,2.0,2.0,1.0,R$ 940 /mês,condomínio R$ 590,IPTU R$ 36,940.0,-22.928051,-43.572791,218.50


In [39]:
tmp_df.rename(columns={'latitude_left':'latitude',
                       'longitude_left':'longitude'},
                       inplace=True)

In [ ]:
# Serialize what I have got
tmp_df.to_csv('./rio1_data_768.csv', encoding='utf-8', index=False)